In [0]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [0]:
import pandas as pd

data = pd.read_csv('drive/My Drive/categorical_less_train.csv')

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [0]:
from h2o.automl import H2OAutoML
import h2o

h2o.init()

train = h2o.H2OFrame(train)

# Выделим целевую переменную из выборки
x = train.columns
y = "price"
x.remove(y)

# Запустим модель для фиксированного числа меньших моделей (max_models = 10)
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train)

# Посмотрим на лидерборд, найденный методом
lb = aml.leaderboard
lb.head(rows=lb.nrows)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphzqwv3sg
  JVM stdout: /tmp/tmphzqwv3sg/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmphzqwv3sg/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_unknownUser_ke7s00
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200604_153207,3772.89,61.4239,3772.89,24.9328,nan
StackedEnsemble_BestOfFamily_AutoML_20200604_153207,3810.55,61.7297,3810.55,24.9196,nan
GBM_1_AutoML_20200604_153207,4205.82,64.8523,4205.82,28.2253,nan
GBM_4_AutoML_20200604_153207,4246.47,65.165,4246.47,26.502,nan
GBM_3_AutoML_20200604_153207,4413.88,66.437,4413.88,28.8101,nan
GBM_2_AutoML_20200604_153207,4495.36,67.0475,4495.36,29.9672,nan
DRF_1_AutoML_20200604_153207,4781.26,69.1467,4781.26,24.1755,0.336173
XGBoost_2_AutoML_20200604_153207,4995.99,70.6823,4995.99,27.84,nan
XGBoost_3_AutoML_20200604_153207,5609.42,74.8961,5609.42,33.2037,nan
XGBoost_1_AutoML_20200604_153207,5653.4,75.1891,5653.4,30.1691,nan


In [0]:
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
pred = aml.predict(test)

In [0]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

print("MSE :",((test["price"] - pred) ** 2).sum() / pred.shape[0])
print("RMSE :",np.sqrt(((test["price"] - pred) ** 2).sum() / pred.shape[0]))
print("R^2 :", r2_score(test["price"].as_data_frame(),pred.as_data_frame()))
print("MAE :", mean_absolute_error(test["price"].as_data_frame(),pred.as_data_frame()))

MSE : 3947.7434341904227
RMSE : 62.83107061152486
R^2 : 0.8970200201861006
MAE : 23.631139129996125


In [0]:
perf = aml.leader.model_performance(test)


In [0]:
perf.show()


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 3947.7434259530573
RMSE: 62.83107054597317
MAE: 23.631139088370787
RMSLE: NaN
R^2: 0.8970200201739906
Mean Residual Deviance: 3947.7434259530573
Null degrees of freedom: 15375
Residual degrees of freedom: 15369
Null deviance: 589448280.038665
Residual deviance: 60700502.91745421
AIC: 170978.30710251734


In [0]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200604_153207

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 642.3735107888094
RMSE: 25.345088494396887
MAE: 14.84647049175607
RMSLE: 0.2860816943309105
R^2: 0.9828491259337216
Mean Residual Deviance: 642.3735107888094
Null degrees of freedom: 10073
Residual degrees of freedom: 10067
Null deviance: 377324431.690402
Residual deviance: 6471270.747686466
AIC: 93734.89541180973

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 3772.889416446276
RMSE: 61.42385055046839
MAE: 24.932760093066427
RMSLE: NaN
R^2: 0.9002865974494124
Mean Residual Deviance: 3772.889416446276
Null degrees of freedom: 35876
Residual degrees of freedom: 35870
Null deviance: 1357529486.0744522
Residual deviance: 135359953.59384304
AIC: 397299.0079256996


In [0]:
# Получим идентификаторы моделей с лидерборда
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Рассмотрим GBM_1 для примера
m = h2o.get_model([mid for mid in model_ids if "GBM_1" in mid][0])  

In [0]:
model_ids

['StackedEnsemble_AllModels_AutoML_20200604_153207',
 'StackedEnsemble_BestOfFamily_AutoML_20200604_153207',
 'GBM_1_AutoML_20200604_153207',
 'GBM_4_AutoML_20200604_153207',
 'GBM_3_AutoML_20200604_153207',
 'GBM_2_AutoML_20200604_153207',
 'DRF_1_AutoML_20200604_153207',
 'XGBoost_2_AutoML_20200604_153207',
 'XGBoost_3_AutoML_20200604_153207',
 'XGBoost_1_AutoML_20200604_153207',
 'GBM_5_AutoML_20200604_153207',
 'GLM_1_AutoML_20200604_153207']

In [0]:
# Выведем показатели гиперпараметров, найденные в результате автоматизации
h2o.get_model(model_ids[2]).summary()


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,327.0,327.0,230375.0,6.0,6.0,6.0,21.0,63.0,48.342506


In [0]:
# Рассмотрим преобразования параметров в результирующей модели
for key in m.params.keys():
  print(key, " : ",m.params[key],end="\n\n")

model_id  :  {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GBM_1_AutoML_20200604_153207', 'type': 'Key<Model>', 'URL': '/3/Models/GBM_1_AutoML_20200604_153207'}}

training_frame  :  {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'automl_training_Key_Frame__upload_ba40956abb6edc567fb2239348948d1.hex', 'type': 'Key<Frame>', 'URL': '/3/Frames/automl_training_Key_Frame__upload_ba40956abb6edc567fb2239348948d1.hex'}}

validation_frame  :  {'default': None, 'actual': None}

nfolds  :  {'default': 0, 'actual': 5}

keep_cross_validation_models  :  {'default': True, 'actual': False}

keep_cross_validation_predictions  :  {'default': False, 'actual': True}

keep_cross_validation_fold_assignment  :  {'default': False, 'actual': False}

score_each_iteration  :  {'default': False, 'actual': False}

score_tree_interval  :  {'default': 0, '